## 🔧 Setup

We'll use LangChain's modern Agent API.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("✅ Agent environment ready!")

---

## 1. Defining Tools

Tools are just Python functions that the AI can call. We use the `@tool` decorator to tell LangChain "this is a tool".

In [ ]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool
def mystery_multiplier(a: int, b: int) -> int:
    """Multiplies two numbers and adds 1. Useful for mystery calculations."""
    return (a * b) + 1

tools = [get_word_length, mystery_multiplier]

print("🛠️ Tools created:")
for t in tools:
    print(f"  - {t.name}: {t.description}")

---

## 2. Creating the Agent

We need a prompt that tells the LLM it has tools available.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to tools. Use them when needed."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create the executor (the runtime for the agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("🤖 Agent initialized!")

---

## 3. Running the Agent

Let's ask questions that REQUIRE tools. The LLM can't know the length of a specific word reliably by token prediction alone, and it definitely doesn't know our "mystery" math rule.

In [ ]:
print("🧪 TEST 1: Word Length")
response = agent_executor.invoke({"input": "How many letters are in the word 'Supercalifragilisticexpialidocious'?"})
print(f"Answer: {response['output']}")

In [ ]:
print("\n🧪 TEST 2: Mystery Math")
response = agent_executor.invoke({"input": "Use the mystery multiplier to calculate 5 times 5."}) 
print(f"Answer: {response['output']}")

## 📝 Summary

You just built an **Agent**! 

- It **reasoned** about which tool to use.
- It **executed** the Python function.
- It **synthesized** the answer.

This is the foundation for building autonomous AI that can search the web, query databases, and perform actions!